# Importation

In [1]:
import numpy as np
import pandas as pd
import json
import xgboost as xgb

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

2023-01-17 00:20:33.328048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-17 00:20:33.491400: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-17 00:20:33.491418: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-17 00:20:34.198798: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

# Création de données

In [4]:
path = "datasets/"

In [22]:
outputs = pd.read_csv(path+"train/train_labels_sent.csv").to_numpy().T[1]
condition = np.logical_and((outputs>70), (outputs<450))
outputs = outputs[condition]
outputs

array([290.16634516, 169.81819677, 144.49971149, ..., 311.68110342,
       340.53874701, 310.07906932])

In [23]:
df = pd.read_csv(path + "train/train_features_sent.csv")
df = df[condition]
df

/tmp/ipykernel_18935/519322580.py:1: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "train/train_features_sent.csv")


,level_0,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_category,building_class,building_height_ft,...,wall_insulation_type,water_heaters,water_heating_energy_source,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity
0,111323,[wood stove or insert],NaN,328.08400,881,NaN,CONCRETE,[],[],13.779528,...,internal,[indeterminate oil boiler],oil,individual,argon or krypton,pvc,double glazing,1.116894,"[est,west]",15.347292
1,265182,[],NaN,360.89240,260,NaN,NaN,[condo],[2 to 11],43.635172,...,internal,[indeterminate gas boiler],gas,individual,argon or krypton,wood,double glazing,1.133072,"[est,west,sud]",17.512631
2,927507,[],NaN,324.80316,573,NaN,WOOD - OTHER,[],[],19.685040,...,internal,[standard gas boiler],gas,individual,dry air,pvc,double glazing,1.084204,"[est,west]",16.779108
3,874620,[],NaN,288.71392,204,NaN,BRICKS,[],[],13.779528,...,internal,[electric storage tank],electricity,individual,NaN,metal with thermal break,triple glazing,0.983338,"[est,north,sud]",12.630033
4,563274,[],NaN,849.73756,374,NaN,GRITSTONE,[individual house],[individual],19.028872,...,internal,[thermodynamic electric hot water (pac or tank)],thermodynamic electric hot water (heat pump or...,individual,dry air,pvc,double glazing,1.084204,"[north,sud]",13.290529
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1010679,84908,[],gas condensing boiler,1591.20740,671,NaN,GRITSTONE,[individual house],[individual],17.060368,...,internal,"[solar water heater,gas condensing boiler]",gas,collective,dry air,wood,double glazing,1.133072,"[est,west]",14.649374
1010680,997676,[],NaN,688.97640,345,< 1 m,INDETERMINED,[],[],26.246720,...,internal,[electric storage tank],electricity,individual,NaN,pvc,double glazing,1.000450,"[west,sud]",11.348055
1010681,311831,[standard gas boiler],NaN,2805.11820,205,NaN,NaN,[individual house],[individual],22.637796,...,internal,[wood boiler],wood,individual,argon or krypton,pvc,double glazing,0.983338,"[est,north,west,sud]",11.982461
1010682,789794,[],NaN,708.66144,451,NaN,STONE,[],[],19.685040,...,non insulated,[indeterminate oil boiler],oil,individual,argon or krypton,pvc,double glazing,1.084204,"[est,north,west,sud]",16.779108


In [24]:
df = df.reset_index(drop=True)

# Data creation function

In [25]:
df.consumption_measurement_date = df.consumption_measurement_date.apply(lambda x: str(int(x[5:7])//3))
df.outer_wall_thickness = df.outer_wall_thickness.apply(lambda x: float(x) if isinstance(x, float) or isinstance(x, int) else float(x.split()[0]))
df["building_period"] = df["building_period"].apply(lambda x: ">1989" if x in ["1989-1999", "2000-2005", '2006-2012', ">2012"] else x)

In [26]:
liste_str = ['balcony_depth', 'building_period', 'building_type', 'building_use_type_description',
 'clay_risk_level', 'has_air_conditioning', 'has_balcony', 'heating_type', 'is_crossing_building',
 'lower_floor_adjacency_type', 'lower_floor_material', 'radon_risk_level', 'solar_heating',
 'solar_water_heating', 'thermal_inertia', 'upper_floor_adjacency_type', 'water_heating_type',
 'window_filling_type', 'window_glazing_type', 'consumption_measurement_date']

In [27]:
liste_int = ['altitude', 'area_code', 'building_height_ft',
       'building_total_area_sqft', 'building_use_type_code', 'building_year', 'living_area_sqft',
       'lowe_floor_thermal_conductivity', 'nb_dwellings',
       'outer_wall_thermal_conductivity', 'post_code', 'upper_floor_thermal_conductivity',
       'window_heat_retention_factor', 'window_thermal_conductivity', 'outer_wall_thickness', 
         'percentage_glazed_surfaced']

In [28]:
df[liste_int] = df[liste_int].fillna(df[liste_int].median())

In [29]:
liste = liste_str + liste_int
df = df[liste]

In [30]:
def transform(df, liste):
    for x in liste:
        list_value = [y for y in df[x].unique() if y is not np.nan]
        dict_value = {y:y for y in list_value}
        df[x] = df[x].map(dict_value)
        df = pd.get_dummies(df, columns=[x], prefix=f'{x}-', prefix_sep='')
    return df

df = transform(df, liste_str)

In [31]:
liste_col = ['altitude', 'area_code', 'building_height_ft',
       'building_total_area_sqft', 'building_use_type_code', 'building_year',
       'living_area_sqft', 'lowe_floor_thermal_conductivity', 'nb_dwellings',
       'outer_wall_thermal_conductivity', 'post_code',
       'upper_floor_thermal_conductivity', 'window_heat_retention_factor',
       'window_thermal_conductivity', 'outer_wall_thickness',
       'percentage_glazed_surfaced', 'balcony_depth-1 <= … < 2',
       'balcony_depth-2 <= … < 3', 'balcony_depth-3 <=', 'balcony_depth-< 1 m',
       'building_period-1949-1970', 'building_period-1970-1988',
       'building_period-<1948', 'building_period->1989',
       'building_period-bad sup', 'building_type-Flat', 'building_type-House',
       'building_type-Multiple dwellings', 'building_type-Non-residential',
       'building_type-Shopping centres',
       'building_use_type_description-Commercial & Other',
       'building_use_type_description-Residential multi-family',
       'building_use_type_description-Residential single-family',
       'clay_risk_level-high', 'clay_risk_level-low', 'clay_risk_level-medium',
       'has_air_conditioning-False', 'has_air_conditioning-True',
       'has_balcony-False', 'has_balcony-True', 'heating_type-collective',
       'heating_type-individual', 'is_crossing_building-90° crossing (weak)',
       'is_crossing_building-all through crossing (weak)',
       'is_crossing_building-crossing east west',
       'is_crossing_building-crossing north south',
       'is_crossing_building-east-west crossing (weak)',
       'is_crossing_building-north-south crossing (weak)',
       'is_crossing_building-not through', 'is_crossing_building-through 90°',
       'is_crossing_building-through all way',
       'lower_floor_adjacency_type-ADJACENT BUILDING',
       'lower_floor_adjacency_type-BURIED_WALL',
       'lower_floor_adjacency_type-FULL_EARTH',
       'lower_floor_adjacency_type-LNC', 'lower_floor_adjacency_type-OUTSIDE',
       'lower_floor_adjacency_type-SANITARY_VACTILE',
       'lower_floor_material-Insulated joist floor',
       'lower_floor_material-Wooden floor on metal joists',
       'lower_floor_material-brick or rubble wall joists',
       'lower_floor_material-concrete slab',
       'lower_floor_material-floor between metal joists with or without infill',
       'lower_floor_material-floor between wooden joists with or without infill',
       'lower_floor_material-floor with or without infill',
       'lower_floor_material-heavy floor, such as clay floor joists, concrete beams',
       'lower_floor_material-joist on metal joists',
       'lower_floor_material-non-differing party floor',
       'lower_floor_material-shingles and infill',
       'lower_floor_material-wooden floor on wooden joists',
       'radon_risk_level-high', 'radon_risk_level-low',
       'radon_risk_level-medium', 'solar_heating-False', 'solar_heating-True',
       'solar_water_heating-False', 'solar_water_heating-True',
       'thermal_inertia-high', 'thermal_inertia-low', 'thermal_inertia-medium',
       'thermal_inertia-very high',
       'upper_floor_adjacency_type-ADJACENT_BUILDING',
       'upper_floor_adjacency_type-EXTERNAL',
       'upper_floor_adjacency_type-INNER_WALL',
       'upper_floor_adjacency_type-LNC', 'water_heating_type-collective',
       'water_heating_type-individual', 'window_filling_type-argon or krypton',
       'window_filling_type-dry air', 'window_glazing_type-double glazing',
       'window_glazing_type-glass block or polycarbonate',
       'window_glazing_type-overglazing', 'window_glazing_type-single glazing',
       'window_glazing_type-triple glazing', 'consumption_measurement_date-0',
       'consumption_measurement_date-1', 'consumption_measurement_date-2',
       'consumption_measurement_date-3', 'consumption_measurement_date-4']

In [20]:
inputs_with_num = df[liste_int].to_numpy()
inputs_with_num.shape

(958425, 16)

In [38]:
inputs_with_str = df[df.columns[16:]].to_numpy()
inputs_with_str.shape

(958425, 82)

# Numerical Model

In [39]:
n = len(outputs)
j = int(n*0.01)
train_features = inputs_with_num[:j]
test_features = inputs_with_num[j:]

train_labels = outputs[:j]
test_labels = outputs[j:]
n

958425

In [40]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [41]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))

In [42]:
model = keras.Sequential([
  normalizer,
    layers.Dropout(0.1),
  layers.Dense(64, activation='relu'),
    layers.Dropout(0.05),
  layers.Dense(32, activation='relu'),
  layers.Dense(16, activation='relu'),
  layers.Dense(1)
])

In [43]:
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam())
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 16)               33        
 ion)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                1088      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                      

In [44]:
epoch = 50
batch = 128*128
save_freq = 10

my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='models/model.{epoch:02d}-{val_loss:.2f}.h5')
]
history = model.fit(
    train_features,
    train_labels,
    validation_split=0.1,
    batch_size=batch,
    verbose=2, epochs=epoch, callbacks=my_callbacks)

Epoch 1/50
1/1 - 2s - loss: 61036.7891 - val_loss: 60790.0039 - 2s/epoch - 2s/step
Epoch 2/50
1/1 - 0s - loss: 61001.3477 - val_loss: 60757.3438 - 125ms/epoch - 125ms/step
Epoch 3/50
1/1 - 0s - loss: 60966.0156 - val_loss: 60725.8438 - 320ms/epoch - 320ms/step
Epoch 4/50
1/1 - 0s - loss: 60932.1836 - val_loss: 60695.3867 - 311ms/epoch - 311ms/step
Epoch 5/50
1/1 - 0s - loss: 60900.0352 - val_loss: 60665.9141 - 305ms/epoch - 305ms/step
Epoch 6/50
1/1 - 0s - loss: 60869.2695 - val_loss: 60637.1758 - 306ms/epoch - 306ms/step
Epoch 7/50
1/1 - 0s - loss: 60836.2422 - val_loss: 60609.1680 - 275ms/epoch - 275ms/step
Epoch 8/50
1/1 - 0s - loss: 60808.0820 - val_loss: 60581.9180 - 313ms/epoch - 313ms/step
Epoch 9/50
1/1 - 0s - loss: 60777.5234 - val_loss: 60555.0820 - 308ms/epoch - 308ms/step
Epoch 10/50
1/1 - 0s - loss: 60747.4492 - val_loss: 60528.2617 - 310ms/epoch - 310ms/step
Epoch 11/50
1/1 - 0s - loss: 60717.9375 - val_loss: 60501.5430 - 217ms/epoch - 217ms/step
Epoch 12/50
1/1 - 0s - lo

In [45]:
link_model_num = "./model_num"
model.save(link_model_num)

INFO:tensorflow:Assets written to: ./model_num/assets


# cat model

In [49]:
n = len(outputs)
j = int(n*0.01)
train_features = inputs_with_str[:j]
test_features = inputs_with_str[j:]

train_labels = inputs_with_str[:j]
test_labels = inputs_with_str[j:]

m = 82

In [50]:
model = keras.Sequential([
  layers.Input(m),
  layers.Dropout(0.1),
  layers.Dense(64, activation='sigmoid'),
  layers.Dropout(0.05),
  layers.Dense(32, activation='sigmoid'),
  layers.Dense(16, activation='sigmoid'),
  layers.Dense(32, activation='sigmoid'),
  layers.Dense(64, activation='sigmoid'),
  layers.Dense(m)
])

In [51]:
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam())
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_4 (Dropout)         (None, 82)                0         
                                                                 
 dense_8 (Dense)             (None, 64)                5312      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 32)                544       
                                                                 
 dense_12 (Dense)            (None, 64)               

In [52]:
epoch = 50
batch = 128*128
save_freq = 10

my_callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath='models/model.{epoch:02d}-{val_loss:.2f}.h5')
]
history = model.fit(
    train_features,
    train_labels,
    validation_split=0.1,
    batch_size=batch,
    verbose=2, epochs=epoch, callbacks=my_callbacks)

Epoch 1/50
1/1 - 2s - loss: 1.6249 - val_loss: 1.5797 - 2s/epoch - 2s/step
Epoch 2/50
1/1 - 0s - loss: 1.5820 - val_loss: 1.5495 - 288ms/epoch - 288ms/step
Epoch 3/50
1/1 - 0s - loss: 1.5519 - val_loss: 1.5251 - 337ms/epoch - 337ms/step
Epoch 4/50
1/1 - 0s - loss: 1.5276 - val_loss: 1.5045 - 221ms/epoch - 221ms/step
Epoch 5/50
1/1 - 0s - loss: 1.5072 - val_loss: 1.4895 - 283ms/epoch - 283ms/step
Epoch 6/50
1/1 - 0s - loss: 1.4928 - val_loss: 1.4754 - 330ms/epoch - 330ms/step
Epoch 7/50
1/1 - 0s - loss: 1.4789 - val_loss: 1.4653 - 283ms/epoch - 283ms/step
Epoch 8/50
1/1 - 0s - loss: 1.4686 - val_loss: 1.4566 - 369ms/epoch - 369ms/step
Epoch 9/50
1/1 - 0s - loss: 1.4602 - val_loss: 1.4484 - 306ms/epoch - 306ms/step
Epoch 10/50
1/1 - 0s - loss: 1.4520 - val_loss: 1.4445 - 326ms/epoch - 326ms/step
Epoch 11/50
1/1 - 0s - loss: 1.4472 - val_loss: 1.4385 - 133ms/epoch - 133ms/step
Epoch 12/50
1/1 - 0s - loss: 1.4414 - val_loss: 1.4331 - 262ms/epoch - 262ms/step
Epoch 13/50
1/1 - 0s - loss: 1.

In [53]:
link_model_str = "./model_str"
model.save(link_model_str)


















































INFO:tensorflow:Assets written to: ./model_str/assets


# Merging models

In [54]:
model_num = tf.keras.models.load_model(link_model_num)
model_num.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization_3 (Normalizat  (None, 16)               33        
 ion)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 64)                1088      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 32)                2080      
                                                                 
 dense_6 (Dense)             (None, 16)                528       
                                                      

In [59]:
model_str = tf.keras.models.load_model(link_model_str)
model_str.layers.pop()
model_str.layers.pop()
model_str.layers.pop()
model_str.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_4 (Dropout)         (None, 82)                0         
                                                                 
 dense_8 (Dense)             (None, 64)                5312      
                                                                 
 dropout_5 (Dropout)         (None, 64)                0         
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 32)                544       
                                                                 
 dense_12 (Dense)            (None, 64)               

In [67]:
concatenate = layers.concatenate(
    [model_num.layers[5].output, model_str.layers[4].output],
    name="concatenated_layer"
)

In [72]:
o = layers.Dense(64, activation='relu')(concatenate)
o = layers.Dense(1, activation='relu')(o)

In [106]:
concatenate_model = keras.models.Model(inputs=[model_num.layers[0].output, model_str.layers[0].output], outputs=[o])


In [81]:
n = len(outputs)
j = int(n*0.01)
train_features = inputs_with_str[:j]
test_features = inputs_with_str[j:]

train_labels = inputs_with_str[:j]
test_labels = inputs_with_str[j:]

m = 82

In [83]:
inputs_with_str[:j].shape

(9584, 82)

In [85]:
inputs_with_num[:j].shape

(9584, 16)

In [101]:
concatenate_model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam())

In [107]:
concatenate_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 16)]         0           []                               
                                                                                                  
 dropout_2 (Dropout)            (None, 16)           0           ['input_4[0][0]']                
                                                                                                  
 input_5 (InputLayer)           [(None, 82)]         0           []                               
                                                                                                  
 dense_4 (Dense)                (None, 64)           1088        ['dropout_2[2][0]']              
                                                                                            

# XGBoost

In [35]:
import sklearn
from sklearn.metrics import explained_variance_score

In [33]:
n = len(outputs)
j = int(n*0.95)
inputs = df.to_numpy()
train_features = inputs[:j]
test_features = inputs[j:]

train_labels = outputs[:j]
test_labels = outputs[j:]

In [43]:
model = xgb.XGBRegressor(eta = 1)
model.fit(train_features, train_labels)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eta=1, eval_metric=None,
             feature_types=None, gamma=None, gpu_id=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor=None, ...)

In [44]:

predictions = model.predict(test_features)
explained_variance_score(test_labels, predictions)

0.5657508759221104